In [ ]:
list.of.packages <- c("tidyverse", "data.table", "dtplyr", "arrow", "dbplyr", "duckdb", "lme4", "lmerTest", "dbscan", "pROC", "caTools", "ez", "matrixStats", "brms" , "glmnet")
new.packages <- list.of.packages[!(list.of.packages %in% installed.packages()[,"Package"])]
if(length(new.packages)) install.packages(new.packages)

library(tidyverse)
library(data.table)
library(dtplyr)
#library(dplyr, warn.conflicts = FALSE)
library(arrow)
library(duckdb)
library(lme4)
library(lmerTest)
library(dbscan)
library(pROC)
library(caTools)
library(ez)
library(matrixStats)
library(brms)
library(glmnet)

In [ ]:
raw_shots <- read_csv("../Data/NBA_Shots_Raw.csv")
player_info <- read_csv("../Data/Player_Info.csv")
player_salary <- read_csv("../Data/Player_Salary.csv")
player_info <- player_info %>% mutate(PLAYER_NAME = paste(First_Name, Surname))
player_salary <- player_salary %>% rename(PLAYER_NAME = Name)
clean_shots <- read_csv("../Data/NBA_Shots_Clean_Example.csv")
common_player_info <- read_csv("../Data/wyatt_basketball/csv/common_player_info.csv")
common_player_info <- common_player_info %>% mutate(PLAYER_NAME = paste(first_name, last_name))
height_2014 <- read_csv("../Data/NBA-Height-Weight/CSVs/Yearly/2014.csv") # https://github.com/simonwarchol/NBA-Height-Weight
height_2014 <- height_2014 %>% rename(PLAYER_NAME = Name)

#clean_shots <- clean_shots %>% mutate(across(where(is.character), ~ str_replace_all(., "Kyle Oquinn", "Kyle O'Quinn")))
#clean_shots <- clean_shots %>% mutate(across(where(is.character), ~ str_replace_all(., "Al Farouq Aminu", "Al-Farouq Aminu")))
#clean_shots <- clean_shots %>% mutate(across(where(is.character), ~ str_replace_all(., "Lamarcus Aldridge", "LaMarcus Aldridge")))
#clean_shots <- clean_shots %>% mutate(across(where(is.character), ~ str_replace_all(., "Oj Mayo", "O.J. Mayo")))
#clean_shots <- clean_shots %>% mutate(across(where(is.character), ~ str_replace_all(., "Cj Miles", "C.J. Miles")))

typos <- list(
    "Time Hardaway Jr" = "Tim Hardaway Jr",
    "Steve Adams" = "Steven Adams",
    "Jose Juan Barea" = "Jj Barea",
    "Glen Rice Jr" = "Glen Rice",
    "Charles Hayes" = "Chuck Hayes", # technically correct but more sources with chuck
    "Ishmael Smith" = "Ish Smith", # as above
    "Patrick Mills" = "Patty Mills", # etc
  #"Luc Mbah" = "Luc Mbah A Moute"
    "Na Nene" = "Nene"
)

replace_strings <- function(df, replacements) {
  # Loop through the replacements
  df %>%
    mutate(across(where(is.character), ~{
      for (pattern in names(replacements)) {
        # Apply only the specific replacements, avoiding duplicate replacement
        . <- str_replace_all(., pattern, replacements[[pattern]])
      }
      .
    }))
}


clean_name <- function(name) {
  name %>%
    str_replace_all("-", " ") %>%
    str_replace_all("'", "") %>%
    str_remove_all("\\.") %>%
    str_to_title()
}



player_info <- player_info %>%
    mutate(PLAYER_NAME = clean_name(PLAYER_NAME))

           
clean_shots <- clean_shots %>%
    mutate(PLAYER_NAME = clean_name(PLAYER_NAME),
          CLOSEST_DEFENDER = clean_name(CLOSEST_DEFENDER))

# shortcut usig slug col
common_player_info <- common_player_info %>%
    mutate(PLAYER_NAME = player_slug) %>%
    mutate(PLAYER_NAME = str_replace_all(PLAYER_NAME, "-", " ")) %>%
    mutate(PLAYER_NAME = str_to_title(PLAYER_NAME))

height_2014 <- height_2014 %>%
    mutate(PLAYER_NAME = clean_name(PLAYER_NAME))


clean_shots <- replace_strings(clean_shots, typos)
player_info <- replace_strings(player_info, typos)
common_player_info <- replace_strings(common_player_info, typos)
height_2014 <- replace_strings(height_2014, typos)

#explicit case because of his name and interactions with the regex
player_info <- player_info %>% mutate(across(where(is.character), ~ str_replace_all(., "Luc Mbah", "Luc Mbah A Moute")))

# more explictit cases (for now)
height_2014 <- height_2014 %>% mutate(across(where(is.character), ~ str_replace_all(., "Jose Barea", "Jj Barea")))

# in common_player_info there is a duplicate row for glen rice, and the younger is not called jr... So by hand:

common_player_info <- common_player_info %>% filter(person_id != 779)

In [ ]:
colnames(clean_shots)
colnames(player_info)
colnames(player_salary)
colnames(height_2014)
#player_statistics %>% count() %>% collect()
#head(raw_shots)
#head(player_info)
#head(player_salary)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 7)

s <- raw_shots %>% 
    left_join(player_info %>% select(Pos, PLAYER_NAME), by = "PLAYER_NAME") %>%
    filter(!is.na(Pos)) %>%
    select(SHOT_DIST, Pos)

ggplot(s, aes(x = SHOT_DIST)) +
    geom_histogram(bins = 30, fill = "blue", color = "black", alpha = 0.5) +
    facet_wrap(~ Pos) +
    labs(title = "Shot Distance Distribution by Position",
       x = "Shot Distance (feet)",
       y = "Frequency") +
    theme_minimal()

In [ ]:
l1 <- raw_shots %>%
    left_join(player_info %>% select(Pos, PLAYER_NAME), by = "PLAYER_NAME", relationship="many-to-many") %>%
    filter(!is.na(Pos)) %>%
    group_by(PLAYER_NAME, Pos) %>% 
    summarize(tot_FGM = sum(FGM), .groups="drop") %>% 
    arrange(desc(tot_FGM))
l2 <- raw_shots %>% 
    filter(FGM == 1) %>% 
    group_by(PLAYER_NAME) %>%
    summarise(
        total_FGM = n(),
        pct_3pt = sum(PTS_TYPE == 3),
        pct_2pt = sum(PTS_TYPE == 2)
    ) %>%
    left_join(player_info %>% select(Pos, PLAYER_NAME, Team, Age), by = "PLAYER_NAME", relationship="many-to-many")

In [ ]:
raw_shots %>% summarise(unique_players = n_distinct(PLAYER_NAME))
player_info %>% summarise(unique_players = n_distinct(PLAYER_NAME))
player_salary %>% summarise(unique_players = n_distinct(PLAYER_NAME))

In [ ]:
#clean_shots %>% 
#    filter(PERIOD <= 3) %>%
#    select(HOME_TEAM, AWAY_TEAM, WIN_LOSE)
#    group_by(

#clean_shots %>% 
#    select(GAME_ID, FINAL_MARGIN, WIN_LOSE, HOME_TEAM, AWAY_TEAM, LOCATION) %>%
#    filter(abs(FINAL_MARGIN) <= 3) %>% #no need to filter for w/l
 #   distinct(GAME_ID, .keep_all=TRUE) %>%
#    summarise(home_win_pct = 100*sum(FINAL_MARGIN > 0 & LOCATION == "H")/sum(LOCATION == "H"))


#clean_shots %>%
#    select(GAME_ID, WIN_LOSE, HOME_TEAM, AWAY_TEAM, LOCATION, PERIOD, PTS_TYPE, SUCCESS) %>%
#    mutate(pts = PTS_TYPE * SUCCESS) %>%
#    filter(PERIOD <= 3) %>%
#    group_by(GAME_ID, LOCATION) %>%
#    summarise(total_pts = sum(pts), .groups = "drop") %>%
#    pivot_wider(names_from = LOCATION, values_from = total_pts, names_prefix = "pts_") %>%
#    summarise(pts_diff_H = pts_H-pts_A)

home_win <- clean_shots %>%
    filter(LOCATION == "H") %>%
    select(GAME_ID, HOME_TEAM, AWAY_TEAM, WIN_LOSE) %>%
    distinct(GAME_ID, .keep_all=TRUE)

clean_shots %>%
    filter(PERIOD <= 3) %>%
    mutate(pts = PTS_TYPE * SUCCESS) %>%
    group_by(GAME_ID, LOCATION) %>%
    summarise(total_pts = sum(pts), .groups = "drop") %>%
    pivot_wider(names_from = LOCATION, values_from = total_pts, names_prefix = "pts_") %>%
    left_join(home_win, by="GAME_ID") %>%
    rename(HOME_RESULT = WIN_LOSE) %>%
    mutate(pts_diff = abs(pts_H - pts_A)) %>%
    filter(pts_diff <= 3) %>%
    summarise(
        home_pct_win = sum(HOME_RESULT == "W")/n(),
        home_wins = sum(HOME_RESULT == "W"),
        total_games = n()
    )

binom.test(x=86, n=161, p=0.5, alternative="greater")

In [ ]:
play_by_play <- open_dataset("../Data/wyatt_basketball/csv/play_by_play.csv", format="csv")

In [ ]:
convert_to_cm <- function(feet_inches) {
  split_height <- strsplit(feet_inches, "-")
  
  feet <- sapply(split_height, function(x) as.numeric(x[1]))
  inches <- sapply(split_height, function(x) as.numeric(x[2]))
  
  cm_height <- (feet * 30.48) + (inches * 2.54)
  
  return(cm_height)
}
convert_to_cm <- Vectorize(convert_to_cm)

In [ ]:
predictors <- c("GAME_ID", "PLAYER_NAME", "CLOSEST_DEFENDER" ,"SHOT_DIST", "PTS_TYPE", "CLOSE_DEF_DIST", "SHOT_CLOCK", "TOUCH_TIME", "PERIOD", "SUCCESS")
colnames(clean_shots)

distinct_players_and_defenders <- union(
    clean_shots %>% distinct(PLAYER_NAME),
    clean_shots %>% distinct(CLOSEST_DEFENDER) %>% rename(PLAYER_NAME = CLOSEST_DEFENDER)
)

player_height_pos <- distinct_players_and_defenders %>%
    left_join(player_info %>% select(PLAYER_NAME, Height, Pos), by="PLAYER_NAME") %>%
    rename(H1 = Height)

player_height_pos <- player_height_pos %>%
    left_join(common_player_info %>% select(PLAYER_NAME, height, position), by="PLAYER_NAME", relationship = "many-to-many") %>%
    rename(H2 = height)
   
player_height_pos <- player_height_pos %>%
    left_join(height_2014 %>% select(PLAYER_NAME, "Height(Feet-Inches)"), by="PLAYER_NAME") %>%
    rename(H3 = "Height(Feet-Inches)") 
# look at overlaps/ number of NAs in different height data now, then choose method to have only one col next


player_height_pos %>%
  group_by(PLAYER_NAME) %>%
  summarise(
    H1_na = any(is.na(H1)),
    H2_na = any(is.na(H2)),
    H3_na = any(is.na(H3)),
    H_all_na = all(is.na(H1) & is.na(H2) & is.na(H3))
  ) %>%
  summarise(
    H1_na_count = sum(H1_na),
    H2_na_count = sum(H2_na),
    H3_na_count = sum(H3_na),
    H_all_na_count = sum(H_all_na)
  )


player_height_pos <- player_height_pos %>%
    mutate(
    H2 = ifelse(!is.na(H2), convert_to_cm(H2), NA),
    H3 = ifelse(!is.na(H3), convert_to_cm(H3), NA),
    HEIGHT = case_when(
        !is.na(H1) ~ H1,
        is.na(H1) ~ rowMeans2(cbind(H2, H3), na.rm = TRUE),
        TRUE ~ NA_real_
    ))


model_data <- clean_shots %>% 
    select(all_of(predictors)) %>%
    left_join(player_height_pos %>% select(PLAYER_NAME, HEIGHT), by="PLAYER_NAME", relationship = "many-to-many") %>%
    rename(SHOOTER_HEIGHT = HEIGHT) %>%
    left_join(player_height_pos %>% select(PLAYER_NAME, HEIGHT) %>% rename(CLOSEST_DEFENDER = PLAYER_NAME), 
        by="CLOSEST_DEFENDER", relationship = "many-to-many") %>%
    rename(DEFENDER_HEIGHT = HEIGHT) %>%
    mutate(SHOOTER_HEIGHT_ADV = SHOOTER_HEIGHT - DEFENDER_HEIGHT) %>%
    filter(PERIOD <= 4) %>%
    mutate(across(c("GAME_ID", "PLAYER_NAME", "CLOSEST_DEFENDER", "PERIOD", "SUCCESS", "PTS_TYPE"), as.factor))


set.seed(0)
train_indices <- sample(1:nrow(model_data), size = 0.70 * nrow(model_data))
train_data <- model_data[train_indices, ]
test_data <- model_data[-train_indices, ]

#dplyr::filter(clean_shots, grepl("Kyle",PLAYER_NAME)) ## found issues with apostrophe in name of Kyle O'Quinn (was Oquinn)

#player_info %>% filter(Team == "LAL") %>% distinct(PLAYER_NAME)

#Robert Sacre not in player_info. Plays for LAL
#Jason Terry not in either. Plays for HOU
#John Salmons not in player_info. Played for NOP and got traded to PHX midseason
#Brandan Wright not in player_info. Played for BOS and got traded to PHX at some point
#Cody Zeller ... Played for Charlotte Bobcats

model_data %>% filter(is.na(DEFENDER_HEIGHT)) %>% distinct(CLOSEST_DEFENDER)

model_data %>% filter(PTS_TYPE == 3) %>% head(10)

In [ ]:
train_data <- train_data %>%
  mutate(
    SHOT_DIST = scale(SHOT_DIST),
    CLOSE_DEF_DIST = scale(CLOSE_DEF_DIST),
    TOUCH_TIME = scale(TOUCH_TIME),
    SHOOTER_HEIGHT_ADV = scale(SHOOTER_HEIGHT_ADV),
    SHOT_CLOCK = scale(SHOT_CLOCK)
  )

test_data <- test_data %>%
  mutate(
    SHOT_DIST = scale(SHOT_DIST),
    CLOSE_DEF_DIST = scale(CLOSE_DEF_DIST),
    TOUCH_TIME = scale(TOUCH_TIME),
    SHOOTER_HEIGHT_ADV = scale(SHOOTER_HEIGHT_ADV),
    SHOT_CLOCK = scale(SHOT_CLOCK)
  )



train_data <- na.omit(train_data)
test_data <- na.omit(test_data)

log_model <- glm(SUCCESS ~ SHOT_DIST + CLOSE_DEF_DIST + TOUCH_TIME + SHOOTER_HEIGHT_ADV + PERIOD + SHOT_CLOCK + PTS_TYPE, 
                 data=train_data, family=binomial(link="logit"))

summary(log_model)


log_mixed_model <- glmer(SUCCESS ~ SHOT_DIST + CLOSE_DEF_DIST + TOUCH_TIME + SHOOTER_HEIGHT_ADV + PERIOD + SHOT_CLOCK + PTS_TYPE
                         + (1 | GAME_ID) + (1 | PLAYER_NAME),
                    data = train_data,
                   family=binomial(link = "logit"),
                        nAGQ = 0)

#summary(log_mixed_model)
#test_data$pred_prob <- predict(log_mm_model, type = "response", newdata=test_data)
#roc(SUCCESS ~ pred_prob, data = test_data)
#summary(log_model)

#brm_model <- brm(
#    formula = SUCCESS ~ SHOT_DIST + CLOSE_DEF_DIST + TOUCH_TIME + SHOOTER_HEIGHT_ADV +
#            SHOT_CLOCK + PTS_TYPE + (1 | PLAYER_NAME),
#    data = train_data,
#    family = bernoulli(),
#    chains = 4,
#    cores = 4,
#    iter = 100
#    )


## the output will explode if you put GAME_ID or PLAYER_NAME in here 
x_train <- model.matrix(SUCCESS ~ SHOT_DIST + CLOSE_DEF_DIST + TOUCH_TIME + SHOOTER_HEIGHT_ADV + PERIOD + SHOT_CLOCK + PTS_TYPE, data = train_data)[, -1]
y_train <- train_data$SUCCESS

x_test <- model.matrix(SUCCESS ~ SHOT_DIST + CLOSE_DEF_DIST + TOUCH_TIME + SHOOTER_HEIGHT_ADV + PERIOD + SHOT_CLOCK + PTS_TYPE, data = test_data)[, -1]
y_test <- test_data$SUCCESS

cv.out <- cv.glmnet(x_train, y_train, family = "binomial", alpha = 1, type.measure = "auc")
plot(cv.out)
lambda_opt <- cv.out$lambda.min

log_lasso_model <- glmnet(x_train, y_train, family = "binomial", alpha=1, lambda=lambda_opt)

In [ ]:
summary(log_model)
summary(log_mixed_model)
summary(log_lasso_model)
log_lasso_model$beta

In [ ]:
model_auroc <- function(model, test_data) {
    pred_prob <- predict(model, newdata = test_data, type="response")
    roc_obj <- roc(test_data$SUCCESS, as.vector(pred_prob), ci=TRUE)
    return(roc_obj)
}

residual_deviance_plot <- function(model, title_str) {
    plot_data <- data.frame(
        fitted = fitted(model),
        residuals = residuals(model, type = "deviance")
    )
    
    resid_plot <- ggplot(plot_data, aes(x = fitted, y = residuals)) +
    geom_point(alpha = 0.3, size = 0.8) +
    geom_hline(yintercept = 0, color = "red", linetype = "dashed") + 
    geom_smooth() +
    labs(
        title = paste0(title_str, ": Residuals vs Fitted"),
        subtitle = "Deviance residuals plotted against predicted probabilities",
        x = "Fitted values (predicted probabilities)",
        y = "Deviance residuals"
    ) +
    theme_bw() +
    theme(
        panel.grid.minor = element_blank(),
        panel.grid.major = element_line(color = "grey90"),
        plot.title = element_text(hjust = 0.5, face = "bold"),
        plot.subtitle = element_text(hjust = 0.5, color = "darkgrey")
    )
    return(resid_plot)
}

model_auroc(log_model, test_data)$ci
model_auroc(log_mixed_model, test_data)$ci
pred_prob <- predict(log_lasso_model, newx = x_test, type="response")
roc(y_test, as.vector(pred_prob), ci=TRUE)$ci

residual_deviance_plot(log_model, "Logistc model")
residual_deviance_plot(log_mixed_model, "Logistic mixed-effects")

In [ ]:
ggplot(clean_shots, aes(x=SHOT_DIST, y=TOUCH_TIME)) + 
    geom_point()
cor(model_data %>% select(SHOT_DIST, SHOT_CLOCK, TOUCH_TIME, CLOSE_DEF_DIST))

In [ ]:
period_mtt <- clean_shots %>%
    group_by(GAME_ID, PERIOD) %>%
    summarise(mean_tt = mean(TOUCH_TIME))

ggplot(period_mtt, aes(x = factor(PERIOD), y = mean_tt)) +
  geom_boxplot() +
  labs(
    x = "Period",
    y = "Mean Touch Time",
    title = "Distribution of Mean Touch Time by Period"
  )

clean_shots %>% 
    filter(PERIOD >= 5) %>%
    summarise(overtime_games = n_distinct(GAME_ID))

period_mtt <- period_mtt %>% filter(PERIOD <= 4) %>% mutate_at(c("PERIOD", "GAME_ID"), as.factor)

summary(aov(mean_tt ~ PERIOD, data=period_mtt))

model <- lmer(mean_tt ~ PERIOD + (1|GAME_ID), data = period_mtt)

anova(model)

summary(model)